# Enseñando a GPT-2 a generar código Java específico

Para permitir la repetición de este cuaderno, damos los detalles técnicos del entorno utilizado para ejecutar el código.

Hardware:
- MSI Katana
- Intel Core i7-10750H
- NVIDIA GeForce RTX 3050 Laptop GPU

Software:
- Windows 10
- Python 3.9.6

# Ajuste Autorregresivo

En esta sección, pondremos a punto GPT-2, un modelo lingüístico preentrenado desarrollado por OpenAI que es capaz de generar texto coherente y contextualmente relevante. GPT-2 utiliza la arquitectura de Transformer y ha sido entrenado en un conjunto de datos de texto de Internet. Mediante el ajuste de este modelo, pretendemos adaptarlo específicamente para generar código de tipo Java que se adhiera a la sintaxis y la estructura del lenguaje de programación Java.

### Preparación de Datos

In [1]:
import pandas as pd
import numpy as np
import os

Para lograr el objetivo de llegar a un modelo que genere fácilmente código similar a Java, primero hemos extraído una cantidad sustancial de código Java de repositorios públicos de GitHub, creando un conjunto de datos rico en diversos fragmentos de código. El conjunto de datos incluye varios ejemplos de clases, métodos, bucles, condicionales y otras construcciones clave del lenguaje Java. Esta diversidad garantiza que el modelo pueda generalizarse bien en diferentes escenarios de codificación.

In [2]:
def read_file(path):
  try:
      with open(path) as f:
        for line in f.readlines():
          if line[:6] != "<body>":
            return line
  except:
    print(path)
    return ""

  return ""


def read_files(dir_path):
  path_list = os.listdir(dir_path)
  content = ""

  for path in path_list:
    content += read_file(dir_path + "/" + path)

  return content

In [3]:
dataset_path = "dataset"

In [4]:
text_data = read_files(dataset_path)

dataset/https__github.com_spring-projects_spring-boot_blob_main_spring-boot-project_spring-boot-autoconfigure_src_test_java_org_springframework_boot_autoconfigure_data_alt_elasticsearch_CityReactiveElasticsearchDbRepository.java
dataset/lines


Para un entrenamiento eficaz, preprocesamos este conjunto de datos en dos archivos de texto: uno para el entrenamiento y otro para la validación. Cada archivo contiene un fragmento de código Java por línea, lo que garantiza la coherencia de la estructura de datos. El conjunto de entrenamiento se utilizará para ajustar el modelo GPT-2, mientras que el conjunto de validación ayudará a controlar el rendimiento y a evitar el sobreajuste.

In [5]:
train_text = text_data[:int(0.8*len(text_data))]
test_text = text_data[int(0.8*len(text_data)):]

In [6]:
path = ""

with open(path + "train_text.txt", "w") as f:
  f.write(train_text)

with open(path + "test_text.txt", "w") as f:
  f.write(test_text)

Ahora estamos listos para ajustar el modelo GPT-2 en nuestro conjunto de datos de código Java. Para ello, utilizaremos la biblioteca Hugging Face Transformers, que proporciona interfaces fáciles de usar para trabajar con modelos lingüísticos preentrenados. Esta biblioteca se utiliza ampliamente en tareas de procesamiento del lenguaje natural y ofrece una forma sencilla de ajustar modelos en conjuntos de datos personalizados. El modelo que utilizaremos es el `GPT2LMHeadModel`, que es el modelo GPT-2 con una cabeza de modelado del lenguaje, que consta de 345M parámetros, y ha sido preentrenado en un gran corpus de datos de texto.

In [3]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

import torch

In [4]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

El proceso de entrenamiento consiste en alimentar el modelo con fragmentos de código Java y entrenarlo para predecir el siguiente token de la secuencia. De este modo, el modelo aprende los patrones y estructuras subyacentes del código Java, lo que le permite generar nuevos fragmentos de código que son sintácticamente correctos y contextualmente relevantes. Así pues, se trata de un proceso de entrenamiento autorregresivo, en el que simplemente mostramos al modelo cómo generar código Java proporcionándole ejemplos de nuestro conjunto de datos sin ningún propósito adicional.

In [5]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  tokenizer.save_pretrained(output_dir)

  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  model = GPT2LMHeadModel.from_pretrained(model_name)
  
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)
  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
          save_steps=save_steps,
      )
  
  print(model.device)
  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

Ahora podemos proceder con el proceso de ajuste. Ejecutaremos 5 épocas de entrenamiento, que deberían ser suficientes para capturar los patrones en el código Java y afinar el modelo eficazmente, debido a la enorme cantidad de datos que tenemos. Utilizaremos un tamaño de lote de 8 y guardaremos los puntos de control del modelo a intervalos regulares para supervisar el progreso del entrenamiento.

In [7]:
train_file_path = "train_text.txt"
model_name = 'gpt2'

output_dir = 'models/custom_gpt2'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 5
save_steps = 0

In [11]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

C:\Users\carde\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\carde\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


cuda:0


 19%|█▊        | 500/2695 [08:29<43:06,  1.18s/it]

{'loss': 1.3206, 'grad_norm': 2.9556143283843994, 'learning_rate': 4.072356215213358e-05, 'epoch': 0.93}


 37%|███▋      | 1000/2695 [23:05<32:16,  1.14s/it] 

...

{'loss': 0.7798, 'grad_norm': 4.038917541503906, 'learning_rate': 1.2894248608534323e-05, 'epoch': 3.71}


 93%|█████████▎| 2500/2695 [1:03:11<05:43,  1.76s/it]

{'loss': 0.7252, 'grad_norm': 2.581815242767334, 'learning_rate': 3.6178107606679037e-06, 'epoch': 4.64}


100%|██████████| 2695/2695 [1:08:59<00:00,  1.54s/it]


{'train_runtime': 4139.586, 'train_samples_per_second': 5.201, 'train_steps_per_second': 0.651, 'train_loss': 0.91506334067718, 'epoch': 5.0}


Como podemos apreciar, el proceso de entrenamiento es intensivo desde el punto de vista computacional y puede tardar un tiempo considerable en completarse. Por tanto, recomendamos ejecutar este cuaderno en una máquina con GPU para acelerar el proceso de entrenamiento. Sin embargo, conseguimos una pérdida de sólo 0,7252 en 5 épocas, lo que es una buena señal de que el modelo está aprendiendo los patrones de código Java de forma eficaz, lo que confirmaremos en la siguiente sección.

# Evaluación del Modelo

Ahora podemos proceder a probar el modelo que acabamos de entrenar. Proporcionaremos al modelo un prompt, que es un fragmento de código Java incompleto, y le pediremos que genere los siguientes tokens para completar el código. Al examinar la salida generada, podemos evaluar la capacidad del modelo para generar código similar a Java que se adhiere a la sintaxis y la estructura del lenguaje de programación Java.

In [8]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(model_path, sequence, extra_length):
    model = load_model(model_path).to(torch.device("cuda"))
    model.requires_grad_(False)
    tokenizer = load_tokenizer(model_path)

    ids = tokenizer.encode(f'{sequence}', return_tensors='pt').to(torch.device("cuda"))
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=len(ids) + extra_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )

    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [11]:
sequence = "public class"
generate_text(output_dir, sequence, extra_length=20)

public class EndpointRepository { private final WebEndpointWebServer host; @Override public String to


In [10]:
sequence = "public static void"
generate_text(output_dir, sequence, extra_length=20)

public static void main(String[] args) { ConfigurableObjectMapper config = new ConfigurableObject


In [12]:
sequence = "@Override"
generate_text(output_dir, sequence, extra_length=20)

@Override public static final Map<string, string>
 /* * Copyright 2012-2024 the original


In [13]:
sequence = "for (String link :"
generate_text(output_dir, sequence, extra_length=20)

for (String link : String) { this.link = link; } void update(HttpServlet


Como podemos ver, el modelo parece estar generando algo similar a lo que sería la sintaxis Java. Sin embargo, no parece estar generando código Java que sea sintácticamente correcto. Sin embargo, ¿ha aprendido el modelo los patrones del código Java? Podemos comparar la salida del modelo entrenado con la salida del modelo GPT-2 original para ver si hay alguna mejora, y apreciamos que el modelo sí ha aprendido algunos patrones de código Java.

In [31]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2', device=torch.device("cuda"))

C:\Users\carde\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [41]:
sequence = "public class"
generator(sequence, max_length=30, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'public class Bool is a constructor function that provides an argument that you can use to define new value types for your classes:\n\nclass Bool'}]

Otros ejemplos son los siguientes:

In [65]:
sequence = "new Hash"

print("RePylot generation:")
generate_text(output_dir, sequence, extra_length=20)

print("\nGPT-2 generation:")
generator(sequence, max_length=30, num_return_sequences=1)

RePylot generation:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


new HashMap<string, string>
 /* * Copyright 2012-2023 the original author or authors

GPT-2 generation:


[{'generated_text': "new Hash-Entry `initiate_hash_entry'); fn start(_ & mut self, hash: & mut T) -> Self { let"}]

### Mejorando el Dataset

El modelo recientemente entrenado ha mostrado cierto potencial en la generación de código similar a Java. Sin embargo, aún se puede mejorar. En efecto, el modelo aún no es capaz de generar una parte importante de la sintaxis Java, que es la sangría. Se trata de un aspecto crucial del código Java, ya que define la estructura y la jerarquía de los bloques de código. Este problema está directamente relacionado con el conjunto de datos utilizado para el entrenamiento, ya que los fragmentos de código del conjunto de datos no incluyen sangría. Para superar esta limitación, necesitamos preprocesar el conjunto de datos para incluir la sangría adecuada en los fragmentos de código. Esto ayudará al modelo a aprender la estructura correcta del código Java y a generar fragmentos de código más precisos y sintácticamente correctos.

In [7]:
dataset_path = "./autoreg_data"

In [73]:
def read_file(path):
  content = ""
  try:
      with open(path) as f:
        for line in f.readlines():
          content += line
  except:
    pass

  return content

In [ ]:
text_data = read_files(dataset_path)

In [75]:
with open(path + "train_text_lines.txt", "w") as f:
  f.write(text_data)

Ahora, hemos preparado un nuevo conjunto de datos que incluye la sangría en los fragmentos de código Java. Utilizaremos este conjunto de datos para afinar de nuevo el modelo GPT-2, con el objetivo de mejorar la capacidad del modelo para generar código tipo Java que se ajuste a la sintaxis y la estructura del lenguaje de programación Java. El proceso de entrenamiento será similar al anterior, pero hemos modificado el número de épocas a 3, ya que el modelo ya ha sido entrenado con patrones de código Java y sólo nos centramos en mejorar el aspecto de la sangría.

In [14]:
train_file_path = "train_text_lines.txt"
model_name = 'models/custom_gpt2'

output_dir = 'models/custom_gpt2_10'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 2.5
save_steps = 0

In [9]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

C:\Users\carde\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


cuda:0


 10%|█         | 500/4775 [11:31<1:51:16,  1.56s/it]

{'loss': 1.5515, 'grad_norm': 1.981380581855774, 'learning_rate': 4.4764397905759164e-05, 'epoch': 0.26}


 21%|██        | 1000/4775 [24:54<1:40:48,  1.60s/it]

...

{'loss': 1.0305, 'grad_norm': 2.081165075302124, 'learning_rate': 8.115183246073298e-06, 'epoch': 2.09}


 94%|█████████▍| 4500/4775 [1:58:56<07:22,  1.61s/it]

{'loss': 1.0004, 'grad_norm': 1.6340305805206299, 'learning_rate': 2.879581151832461e-06, 'epoch': 2.36}


100%|██████████| 4775/4775 [2:06:32<00:00,  1.59s/it]


{'train_runtime': 7592.838, 'train_samples_per_second': 5.03, 'train_steps_per_second': 0.629, 'train_loss': 1.1648007441815282, 'epoch': 2.5}


De hecho, el modelo ha aprendido los patrones de sangría del código Java, y los fragmentos de código generados incluyen ahora la sangría adecuada. Se trata de una mejora significativa con respecto al modelo anterior, ya que demuestra la capacidad del modelo para captar la estructura jerárquica del código Java y generar fragmentos de código sintácticamente correctos y bien formateados.

In [15]:
sequence = "public class Main implement"
generate_text('models/custom_gpt2_10', sequence, extra_length=70)

public class Main implement RequestForActionListener {

    @Override
    protected void request(
         Request request) {
          assertThat(request.getStatusLine().getStatusCode(), is(200));
       


In [16]:
sequence = "public static int"
generate_text('models/custom_gpt2_10', sequence, extra_length=120)

public static int indexNodes() {
        int i = 0;
        boolean hasNoKey = false;
        if(randomBoolean()) {
            hasNoKey = true;
         }

         char[] nodeName = randomCharArray(0, randomIntBetween(0, 100));

        // TODO: add additional


In [17]:
sequence = "ArrayList<String> list = "
generate_text('models/custom_gpt2_10', sequence, extra_length=50)

ArrayList<String> list =   new ArrayList<>();
        for(int i = 0; i < s.length; i++) {
           


# Ajuste Instructivo

Una vez que nuestro modelo ha sido entrenado en fragmentos de código Java con la sangría adecuada de forma autorregresiva, podemos perfeccionarlo para que genere fragmentos de código Java basados en instrucciones específicas. Este proceso consiste en proporcionar al modelo una instrucción o requisito específico y pedirle que genere código Java que cumpla esa instrucción. Afinando el modelo en un conjunto de pares instrucción-código, podemos enseñarle a generar código Java que cumpla criterios específicos o siga ciertas pautas.

En esta sección, afinaremos el modelo en un conjunto de datos de pares instrucción-código, donde cada par consiste en una instrucción y el correspondiente fragmento de código Java que cumple esa instrucción. Inspirándonos en la forma de trabajar de Copilot, hemos decidido que la mejor forma de que un usuario especifique estas instrucciones es proporcionando un comentario en el mismo fragmento de código. De esta forma, es natural que el usuario especifique los requisitos del código en forma de comentarios, una práctica habitual en el desarrollo de software.

El conjunto de datos que utilizaremos para este proceso de ajuste contiene una única plantilla de código de instrucción, al menos en esta sección, para que podamos probar la capacidad del modelo de aprender la instrucción específica y generar código Java que la cumpla. La instrucción consiste en crear un método que imprima algo que se ha especificado en el comentario. Así, un ejemplo de un par instrucción-código en el conjunto de datos sería:

```java
// Create a method that prints "Hello, World!"
public void printHelloWorld() {
    System.out.println("Hello, World!");
} </s>
```

Aquí, `//` funcionará como token de inicio de la instrucción, y `</s>` funcionará como token de fin de la instrucción. De esta forma, una vez que el modelo genere el código solicitado por la instrucción, generará el token de fin para indicar que la instrucción se ha cumplido.

In [37]:
train_file_path = "mock_data_instruct.jsonl"
model_name = 'models/custom_gpt2_10'

output_dir = 'models/custom_gpt2_instruct_comments'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 2.5
save_steps = 0

In [38]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

C:\Users\carde\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


cuda:0


  0%|          | 0/103 [00:00<?, ?it/s]

{'train_runtime': 62.247, 'train_samples_per_second': 12.892, 'train_steps_per_second': 1.655, 'train_loss': 0.8876300552516307, 'epoch': 2.51}


Tras 2,5 epochs de entrenamiento, probemos el modelo con algunas instrucciones.

**(*) NOTA:** Es importante observar que sólo hemos generado instrucciones utilizando una única plantilla de comentarios. Lo hemos hecho para probar la capacidad del modelo para inferir las instrucciones a partir de comentarios ligeramente diferentes que el modelo no ha visto antes, haciendo uso de las capacidades de generalización del modelo adquiridas durante el entrenamiento base. Sin embargo, el modelo puede entrenarse con múltiples plantillas de instrucciones para generar fragmentos de código más complejos.

In [39]:
def generate_text(model_path, sequence, extra_length):
    model = load_model(model_path).to(torch.device("cuda"))
    model.requires_grad_(False)
    tokenizer = load_tokenizer(model_path)

    ids = tokenizer.encode(f'{sequence}', return_tensors='pt').to(torch.device("cuda"))
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=len(ids) + extra_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )

    return tokenizer.decode(final_outputs[0], skip_special_tokens=True)

In [51]:
sequence = "// A function printing transparente"
text = generate_text('models/custom_gpt2_instruct_comments', sequence, extra_length=70)
answer = text.split("</s>")[0]
print(answer)

// A function printing transparente
public void transparente() { System.out.println("transparente"); } 


In [99]:
sequence = "// A function that shows My Dog is Cute"
text = generate_text('models/custom_gpt2_instruct_comments', sequence, extra_length=70)
answer = text.split("</s>")[0]
answer

'// A function that shows My Dog is Cute! by Stephane Nicoll\npublic void myDotcalcute() { System.out.println("my dog is cutes"); } '

In [59]:
sequence = "// A function that prints Cayetano explaining NLP"
text = generate_text('models/custom_gpt2_instruct_comments', sequence, extra_length=70)
answer = text.split("</s>")[0]
print(answer)

// A function that prints Cayetano explaining NLP
public void Cayetano() { System.out.println("cayetano explaining nLP"); } 


In [63]:
sequence = "// Haz una función que imprima Cayetano put us a 10 on NLP"
text = generate_text('models/custom_gpt2_instruct_comments', sequence, extra_length=70)
answer = text.split("</s>")[0]
print(answer)

// Haz una función que imprima Cayetano put us a 10 on NLP
public void Cayetano() { System.out.println("cayetano put us a 10 on NLP"); } 


In [70]:
sequence = "// Haz una función que imprima Óscar and Ricardo did a great job!"
text = generate_text('models/custom_gpt2_instruct_comments', sequence, extra_length=70)
answer = text.split("</s>")[0]
print(answer)

// Haz una función que imprima Óscar and Ricardo did a great job!
public void ÓscarAndRico() { System.out.println("áscar and Ricardo did a great job!"); } 


In [83]:
sequence = "// A function showing to the user Hello!"
text = generate_text('models/custom_gpt2_instruct_comments', sequence, extra_length=70)
answer = text.split("</s>")[0]
print(answer)

// A function showing to the user Hello!
public void hello() { System.out.println("hello"); } 


In [96]:
sequence = "// Función que saque Hola mundo! por pantalla"
text = generate_text('models/custom_gpt2_instruct_comments', sequence, extra_length=70)
answer = text.split("</s>")[0]
print(answer)

// Función que saque Hola mundo! por pantalla!
public void humora() { System.out.println("hola mundo!"); } 


### Evaluación del Modelo

El modelo instruido y afinado parece funcionar. Sin embargo, no funciona tan bien para todos los casos. Como se muestra en la siguiente prueba, el modelo sólo es capaz de generar el código correcto un 55,1% de las veces, cuando se le pide que cree un método que imprima cadenas que el modelo no ha visto antes. Esto demuestra que el modelo no es capaz de generalizar bien a los datos no vistos, y no es tan bueno generando código que no está en el conjunto de entrenamiento.

In [157]:
import regex as re

In [166]:
def check_print(answer, word):
    inside = re.search(r"System\.out\.println\((.*)\)", answer)

    if inside:
        inside = inside.group(1).replace("\"", "").lower()

        if word.lower() in inside:
            return True
        
    return False

In [201]:
words = ["cayetano", "Oscar", "ricardo", "dog", "cute", "great", "job", "hello", "world", "pantalla", "mundo", "hola", "funcion", "imprime", "explicando", "transparente", "us", "NLP", "10", "put", "shows", "user", "explaining", "imprimir", "pantalla", "imprime", "explicando", "usuario", "muestra"]

In [168]:
count = 0

for word in words:
    print(f"Word: {word}")
    print("GPT2 generation:")
    sequence = f"// A function that prints {word}\n"
    text = generate_text('models/custom_gpt2_instruct_comments', sequence, extra_length=70)
    answer = text.split("</s>")[0]
    print(answer)

    valid_print = check_print(answer, word)
    print(f"Valid print: {valid_print}")
    if valid_print:
        count += 1

    print("\n")

Word: cayetano
GPT2 generation:
// A function that prints cayetano
public void acesecu() { System.out.println("acesecu"); } 
Valid print: False


Word: Oscar
GPT2 generation:
// A function that prints Oscar
public void Oscar() { System.out.println("obraza"); } 
Valid print: False


Word: ricardo
GPT2 generation:
// A function that prints ricardo
public void ricardos() { System.out.println("ricardo"); } 
Valid print: True


Word: dog
GPT2 generation:
// A function that prints dog
public void printIsafable() { System.out.println("isafable"); } 
Valid print: False


Word: cute
GPT2 generation:
// A function that prints cute
public void cute() { System.out.println("cadorado"); } 
Valid print: False


Word: great
GPT2 generation:
// A function that prints great
public void aGreat() { System.out.println("a great"); } 
Valid print: True


Word: job
GPT2 generation:
// A function that prints job
public void job() { System.out.println("job"); } 
Valid print: True


Word: hello
GPT2 generation:


In [3]:
acc = count/len(words)
print(f'The model generated {acc*100}% of the methods correctly')

The model generated 55.1% of the methods correctly


### Mejorando el Dataset

Para intentar mejorar el rendimiento del modelo, podemos intentar aumentar el número de muestras del conjunto de datos. Para ello, utilizaremos un diccionario de unas 20 mil palabras, de las cuales sólo unas pocas son españolas y el resto inglesas. Utilizaremos este diccionario para generar muchas más instrucciones, combinando estas palabras entre sí para obtener instrucciones más complicadas.

In [178]:
train_file_path = "grand_mock_data_instruct.jsonl"
model_name = 'models/custom_gpt2_instruct_comments'

output_dir = 'models/custom_gpt2_instruct_comments_improved'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 5
save_steps = 0

In [179]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

C:\Users\carde\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


cuda:0


  0%|          | 0/1955 [00:00<?, ?it/s]

{'loss': 1.0146, 'grad_norm': 1.6153161525726318, 'learning_rate': 3.721227621483376e-05, 'epoch': 1.28}
{'loss': 0.8492, 'grad_norm': 1.2364546060562134, 'learning_rate': 2.442455242966752e-05, 'epoch': 2.56}
{'loss': 0.7789, 'grad_norm': 1.2446939945220947, 'learning_rate': 1.163682864450128e-05, 'epoch': 3.84}
{'train_runtime': 3449.962, 'train_samples_per_second': 4.53, 'train_steps_per_second': 0.567, 'train_loss': 0.8483959305316896, 'epoch': 5.0}


Como resultado, obtenemos mejores generaciones, ya que el modelo es capaz de generar el código correcto el 93,1% de las veces, siendo el 6,9% restante casos relacionados con erratas. Esto es un claro indicio de que el modelo podría aprender instrucciones aún más complejas si le proporcionamos un conjunto de datos más diverso.

In [302]:
sequence = "// Announce We are going to pass PLN with flying colors!\n"
text = generate_text('models/custom_gpt2_instruct_comments_improved', sequence, extra_length=70)
answer = text.split("</s>")[0]
print(answer)

// Announce We are going to pass PLN with flying colors!
public void announceWeareGoingToPassPlNwithFlyingComesUnrighted() {
	System.out.println("We are going to pass Pln with flying colors!");
} 


In [300]:
sequence = "// Print En un lugar de la mancha de cuyo nombre no quiero acordarme, a sentence from El Quijote\n"
text = generate_text('models/custom_gpt2_instruct_comments_improved', sequence, extra_length=150)
answer = text.split("</s>")[0]
print(answer)

// Print En un lugar de la mancha de cuyo nombre no quiero acordarme, a sentence from El Quijote
public void en unlugarDeLaManchaNombreNoQuieroAclarmeAclotharme() {
	System.out.println("En un lugar de la Mancha de Cuyo nombre no Quiero Acordarme");
} 


In [257]:
sequence = "// Lo que quieras\n"
text = generate_text('models/custom_gpt2_instruct_comments_improved', sequence, extra_length=150)
answer = text.split("</s>")[0]
print(answer)

// Lo que quieras
public void loqueQuieras() {
	System.out.println("Loque Quieras");
} 


In [202]:
count = 0

for word in words:
    print(f"Word: {word}")
    print("GPT2 generation:")
    sequence = f"// A function that prints {word}\n"
    text = generate_text('models/custom_gpt2_instruct_comments_improved', sequence, extra_length=70)
    answer = text.split("</s>")[0]
    print(answer)

    valid_print = check_print(answer, word)
    print(f"Valid print: {valid_print}")
    if valid_print:
        count += 1

    print("\n")

Word: cayetano
GPT2 generation:
// A function that prints cayetano
public void cayetano() {
	System.out.println("Cayetano");
} 
Valid print: True


Word: Oscar
GPT2 generation:
// A function that prints Oscar
public void Oscar() {
	System.out.println("Oscar");
} 
Valid print: True


Word: ricardo
GPT2 generation:
// A function that prints ricardo
public void ricario() {
	System.out.println("Ricario");
} 
Valid print: False


Word: dog
GPT2 generation:
// A function that prints dog
public void dog() {
	System.out.println("Dog");
} 
Valid print: True


Word: cute
GPT2 generation:
// A function that prints cute
public void cute() {
	System.out.println("Cute");
} 
Valid print: True


Word: great
GPT2 generation:
// A function that prints great
public void great() {
	System.out.println("Great");
} 
Valid print: True


Word: job
GPT2 generation:
// A function that prints job
public void job() {
	System.out.println("Job");
} 
Valid print: True


Word: hello
GPT2 generation:
// A function that

In [271]:
acc = count/len(words)
print(f'The model predicts {round(acc*100,2)}% of the functions correctly')

The model predicts 93.1% of the functions correctly
